In [1]:
import Pkg; 
Pkg.add("GLPK")
Pkg.add("JuMP")
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Cbc")

In [2]:
const DATA_DIR = joinpath(@__DIR__, "data");



import DataFrames
import CSV
import Random
import GLPK
import Cbc
import Statistics
using JuMP

In [3]:
    ########### Parsing the CSV data: #######################

    csv_df = CSV.read(joinpath(DATA_DIR, "courseData.csv"), DataFrames.DataFrame)

    ProfsSet = Set{String}(skipmissing(csv_df[!, 1])) # set of all profs
    numProfs = length(ProfsSet)
    CoursesSet = Set{String}() # set of all courses
    profToCourseDict = Dict{String, Set{String}}() # Dict of the format {ProfName => {ProfCourse1, ProfCourse2, ...}}

    for i in 1:7 # The range 1:7 is based on the current format of the csv
        currProf = csv_df[i, 1]
        currCourses = Set{String}()
        for j in 2:4
            if ~ ismissing(csv_df[i, j])
                push!(currCourses, csv_df[i, j])
                push!(CoursesSet, csv_df[i, j] )
            end        
        end
        profToCourseDict[currProf] = currCourses
    end

    numCourses = length(CoursesSet) # Based on number of courses in csv

    ############ Getting Course Timings ##############

    courseTimingsDict = Dict{String, Vector{Any}}()

    for i in 1:numProfs
        currProf = csv_df[i, 1]
        days = csv_df[i, 5]
        startTime = csv_df[i, 6]
        EndTime = csv_df[i, 7]
        for course in profToCourseDict[currProf]
            courseTimingsDict[course] = [days, startTime, EndTime]
        end    
    end

    # courseTimingsDict

    ####### Cap on Number of Seats in a Course ########

    # TODO: THIS WILL CAUSE THE PROGRAM TO FAVOUR COURSES THAT HAVE A HIGHER NUMBER OF SEATS
    # SHOULD I FACTOR THIS INTO THE PROGRAM SOMEHOW? (Maybe each prof can offer a max number of seats? I'm not sure yet)

    courseCapDict = Dict{String, Int32}()

    for i in 1:numCourses
        courseCapDict[csv_df[i,9]] = csv_df[i,10]
    end

    # courseCapDict


    #############################################################


    #############################################################

    ##### Creating Course Timings Dictionary #######

    # courseTimeBool is a dictionary of the format:
    # { (course, timestep) => 1 if course takes place at timestep and 0 otherwise}
    # An example of a timestep is W1335 which refers to Wednesday at 1:35pm


    courseTimeBool = Dict{Tuple{String, String}, Int32}()

    discreteTimes = []
    TimeIdsSet = Set()
    for k in 7:23
        for j in 0:5
            for i in [0, 5]
                currTime = "$k$(j)$(i)"
                currTime = parse(Int64, currTime)
                push!(discreteTimes, currTime)
            end
        end
    end

    days = "MTWRF"
    for currTime in discreteTimes
        for day in days
            currTimeString = string(currTime)
            currTimeId = "$(day)$(currTimeString)"
            push!(TimeIdsSet, currTimeId)
            for course in CoursesSet
                if day in courseTimingsDict[course][1]
                    startTime = courseTimingsDict[course][2]
                    endTime = courseTimingsDict[course][3]
                    # Not strictly greater than or less than because
                    # courses can take place back to back.
                    afterStartTime = currTime > startTime
                    beforeEndTime = currTime < endTime

                    if afterStartTime & beforeEndTime
                        courseTimeBool[(course, currTimeId)] = 1
                    end
                end
            end
        end
    end

    # For all the keys that did not get a value of 1, I'm setting them
    # to 0
    # This is essentially what a defaultdict does, but defaultdicts
    # don't work well with optimizers so I'm doing this "manually" here instead
    for timeId in TimeIdsSet
        for course in CoursesSet
            if ~ ((course, timeId) in keys(courseTimeBool))
                courseTimeBool[(course, timeId)] = 0
            end
        end
    end


In [4]:
    ############ Helper Function ####################################

    # Function that returns a random list of integers of length "len" with the sum of the integers equal to
    # "fixedSum" to reflect how a student distributes a fixed number of points among the courses being offered
    function getRandomFixedSum(len, fixedSum)
        res = Int64[]
        currSum = 0

        # adding a random integer for each course in the list
        for i in 1:len
            # if fixedSum reached, the rest of the  values will be 0
            if currSum >= fixedSum
                push!(res, 0)
            else
                # this if condition ensures that each student uses all the points available to him
                # by spending all remaining points on the last course in the list
                if i == len
                    push!(res, (fixedSum-currSum))
                else
                    currRand = rand(0:(fixedSum-currSum)) # ensures student does not spend more points than they have left
                    push!(res, currRand)
                    currSum += currRand
                end
            end        
        end

        # shuffling the list to ensure no bias towards any specific course (will need to make this more reflective 
        # of reality later)
        Random.shuffle!(res)

        # Since 0 represents that a student does not want to take a course, this discourages a student being assigned to
        # courses that they do not want
        replace!(res, 0 => -1000)
        return res
    end 

    #############################################################



getRandomFixedSum (generic function with 1 method)

In [5]:
function main(numStudents, solver, courseRatingDict)

    # courseTimeBool

    #############################################################
    if solver == "SCIP"
        model = Model(GLPK.Optimizer)
    elseif solver == "GLPK"
        model = Model(GLPK.Optimizer)
    elseif solver == "Cbc"
        model = Model(Cbc.Optimizer)
        set_optimizer_attribute(model, "logLevel", 0)
    else
        model = Model(GLPK.Optimizer)
        
        end            
        

    ########## DECISION VARIABLES ########################

    @variable(model, coursesBool[course in CoursesSet], Bin);

    @variable(model, studentAssignments[student in 1:numStudents, course in CoursesSet], Bin);

    ####### Trick for Linearizing Quadratic Terms!!! ###############

    # The essense of the trick is it ensures that : z[student, course] = studentAssignment[student, course] * coursesBool[course]
    # using the 4 constraints below
    # got the trick from this website: https://orinanobworld.blogspot.com/2010/10/binary-variables-and-quadratic-terms.html

    @variable(model, z[student in 1:numStudents, course in CoursesSet], Bin);

    for student in 1:numStudents
        for course in CoursesSet
            @constraint(model, z[student, course] <= studentAssignments[student, course])
            @constraint(model, z[student, course] >= 0) # this isnt really needed in our case
            @constraint(model, z[student, course] <= coursesBool[course])
            @constraint(model, z[student, course] >= (coursesBool[course] - (1-studentAssignments[student, course])))
        end
    end

    #################################################################

    ###### CONSTRAINTS #############################################

    # Constraint: min and max number of electives given to a student
    for student in 1:numStudents
        @constraint(model, 1 <= sum(z[student, course] for course in CoursesSet) <= 2)
    end

    # Constraint: Cap on # of students in a course
    for course in CoursesSet
        @constraint(model, sum(z[student, course] for student in 1:numStudents) <= courseCapDict[course]) 
    end


    # Constraint: Time Conflict constraint for Students
    for timeId in TimeIdsSet
        for student in 1:numStudents
            @constraint(model, sum([courseTimeBool[(course, timeId)]*z[student, course] for course in CoursesSet]) <= 1)
        end
    end

    # Constraint: Time Conflict constraint for Profs (Uncomment when Profs teach courses at different times) 
    # (currently each prof teaches only one course so this isnt needed.)

    # for timeId in TimeIdsSet
    #     for prof in ProfsSet
    #         @constraint(model, sum([courseTimeBool[(course, timeId)] for course in profToCourseDict[prof]]) <= 1)
    #     end
    # end

    # Contraint: at most 6 courses to be chosen (replaced in favour of the next constraint)
    # @constraint(model, sum([coursesBool[course] for course in CoursesSet]) <= 6)

    # Constraint: exactly one course from one prof
    for prof in ProfsSet
        @constraint(model, sum([coursesBool[course] for course in profToCourseDict[prof]]) == 1)
    end

    #################################################################

    # Objective
    @objective(model, Max, sum([z[student, course]*courseRatingDict[student, course] for student in 1:numStudents, course in CoursesSet]));

    optimize!(model)

#     println(solution_summary(model))

    ### DISPLAYING THE RESULTS ##############

    for course in CoursesSet
        if getvalue(coursesBool[course]) > 0.5 # tol of 0.5
#             println("Students assigned to $(course) are:")
            currStudents = []
            for student in 1:numStudents
                if getvalue(studentAssignments[student, course]) > 0.5 # tol of 0.5
                    push!(currStudents, student)
                end
            end
#             println(currStudents)
        end
    end

    #################################################################

    ### DISPLAYING THE RESULTS ##############

    for student in 1:numStudents
        currCourses = String[]
        for course in CoursesSet
            if getvalue(coursesBool[course]) > 0.5 # tol of 0.5
                if getvalue(studentAssignments[student, course]) > 0.5 # tol of 0.5
                    push!(currCourses, course)
                end
            end
        end
#         println("Student $(student): $(currCourses)")
    end
    
    return (round(solve_time(model), digits=2), getobjectivevalue(model))
    #################################################################
end

main (generic function with 1 method)

In [ ]:
######################### RUNTIME ANALYSIS ###########################

timeTakenList = []
for numStudents in 30:5:150
    println("Number of Students: $(numStudents)")
    
    ############ Creating Course Rating Dictionary ##############
    courseRatingDicts = []
    for _ in 1:5
        courseRatingDict = Dict{Tuple{Int64, String}, Int64}()
        for student in 1:numStudents
            currRatings = getRandomFixedSum(numCourses,10)
            for (index, course) in enumerate(CoursesSet)
                courseRatingDict[(student, course)] = currRatings[index]
            end
        end
        push!(courseRatingDicts, courseRatingDict)
    end
    
#     println(courseRatingDicts)
    
    
    ##############################################################
    
    for currentSolver in ["Cbc", "GLPK"]
        currTimes = []
        currObjectiveValues = []
        for j in 1:5
            currSolveTime, currObjectiveValue = main(numStudents, currentSolver, courseRatingDicts[j])
            push!(currTimes, currSolveTime)
            push!(currObjectiveValues, currObjectiveValue)
        end

        currMeanSolveTime = round(Statistics.mean(currTimes), digits=2)

        currMeanObjectiveValue = round(Statistics.mean(currObjectiveValues), digits=2)
        currElem = (currentSolver, numStudents, currMeanSolveTime, currTimes, currMeanObjectiveValue, currObjectiveValues)
        println(currElem)
        push!(timeTakenList, currElem)
        
    end
    println()
end 
        

Number of Students: 30
("Cbc", 30, 0.25, Any[0.3, 0.31, 0.33, 0.15, 0.14], -1235.4, Any[-843.0, -819.0, -2858.0, -1834.0, 177.0])
("GLPK", 30, 0.9, Any[0.86, 0.99, 0.97, 0.83, 0.86], -1235.4, Any[-843.0, -819.0, -2858.0, -1834.0, 177.0])

Number of Students: 35
("Cbc", 35, 0.44, Any[0.61, 0.45, 0.22, 0.51, 0.41], -2035.4, Any[-3823.0, -832.0, -1823.0, -1835.0, -1864.0])
("GLPK", 35, 1.65, Any[1.39, 1.56, 1.27, 2.07, 1.95], -2035.4, Any[-3823.0, -832.0, -1823.0, -1835.0, -1864.0])

Number of Students: 40
("Cbc", 40, 1.27, Any[1.13, 0.58, 4.11, 0.25, 0.26], -4795.8, Any[-5800.0, -6803.0, -4806.0, -3783.0, -2787.0])
("GLPK", 40, 2.45, Any[2.71, 1.79, 4.27, 1.66, 1.84], -4795.8, Any[-5800.0, -6803.0, -4806.0, -3783.0, -2787.0])

Number of Students: 45
("Cbc", 45, 1.11, Any[1.13, 1.08, 1.14, 0.32, 1.9], -3582.2, Any[-3748.0, -3802.0, -4764.0, -1780.0, -3817.0])
("GLPK", 45, 3.65, Any[6.72, 3.51, 2.7, 2.0, 3.34], -3582.2, Any[-3748.0, -3802.0000000000005, -4764.0, -1780.0, -3817.0])

Number 